In [ ]:
from datetime import datetime
  
from pytz import timezone
tz = timezone('EST')
print(datetime.now(tz)) 

2022-05-10 08:52:52.911725-05:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Add this in a Google Colab cell to install the correct version of Pytorch Geometric.
!pip uninstall -y torch
!pip install torch==1.10.1+cu111 torchvision==0.11.2+cu111 torchaudio==0.10.1 -f https://download.pytorch.org/whl/torch_stable.html
#print(torch.__version__)
def format_pytorch_version(version):
  return version.split('+')[0]

import torch
import numpy as np
import gc

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric 
!pip install torch ninja
!pip install -U MinkowskiEngine --install-option="--blas=openblas" -v --no-deps

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████▌                   | 834.1 MB 6.8 MB/s eta 0:03:12tcmalloc: large alloc 1147494400 bytes == 0x38de6000 @  0x7ff697f8b615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |███████████████▉                | 1055.7 MB 1.4 MB/s eta 0:13:18tcmalloc: large alloc 1434370048 bytes == 0x7d43c000 @  0x7ff697f8b615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████████            | 1336.2 MB 1.4 MB/s eta 0:09:23tcmalloc: large alloc 1792966656 bytes =

In [ ]:
tz = timezone('EST')
print(datetime.now(tz)) 

2022-05-10 09:14:27.344583-05:00


In [ ]:
import torch
from torch_geometric.datasets import S3DIS
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

#dataset = S3DIS(root='/content/drive/MyDrive/CS674_Project/Dataset/')

True
Tesla K80


In [ ]:
from torch_geometric.datasets import S3DIS
train_dataset = S3DIS(
    '/content/',
    test_area=5, 
    train=True)

test_dataset = S3DIS(
    '/content/',
    test_area=5, 
    train=False)

Extracting /content/indoor3d_sem_seg_hdf5_data.zip
Processing...
Done!


In [ ]:
bs = 5
from torch_geometric.loader import DataLoader
loader = DataLoader(train_dataset, batch_size=bs)
from torch_geometric.loader import DataLoader
testloader = DataLoader(test_dataset, batch_size=bs)

In [ ]:
import MinkowskiEngine as ME
import torch
import MinkowskiEngine.MinkowskiFunctional as MF
import MinkowskiEngine.MinkowskiNonlinearity as MN


In [ ]:
class ResUNet(ME.MinkowskiNetwork):

    def __init__(self, in_nchannel, out_nchannel, D):
        super(ResUNet, self).__init__(D)
        
        self.conv_n_1 = torch.nn.Sequential(
            ME.MinkowskiConvolution(
                in_channels=in_nchannel,
                out_channels=32,
                kernel_size=1,
                stride=1,
                dimension=D),
            ME.MinkowskiBatchNorm(32),   # maybe not needed 
        )
        
        self.res_block_1 = torch.nn.Sequential(
            ME.MinkowskiBatchNorm(32),
            MN.MinkowskiReLU(),
            ME.MinkowskiConvolution(
                in_channels=32,
                out_channels=32,
                kernel_size=3,
                stride=1,
                dimension=D),
            ME.MinkowskiBatchNorm(32),
            MN.MinkowskiReLU(),
            ME.MinkowskiConvolution(
                in_channels=32,
                out_channels=32,
                kernel_size=3,
                stride=1,
                dimension=D))
        
        self.conv_1 = torch.nn.Sequential(
            ME.MinkowskiConvolution(
                in_channels=32,
                out_channels=64,
                kernel_size=3,
                stride=2,
                dimension=D))
        
        self.res_block_2 = torch.nn.Sequential(
            ME.MinkowskiBatchNorm(64),
            MN.MinkowskiReLU(),
            ME.MinkowskiConvolution(
                in_channels=64,
                out_channels=64,
                kernel_size=3,
                stride=1,
                dimension=D),
            ME.MinkowskiBatchNorm(64),
            MN.MinkowskiReLU(),
            ME.MinkowskiConvolution(
                in_channels=64,
                out_channels=64,
                kernel_size=3,
                stride=1,
                dimension=D))
        
        self.conv_2 = torch.nn.Sequential(
            ME.MinkowskiConvolution(
                in_channels=64,
                out_channels=128,
                kernel_size=3,
                stride=2,
                dimension=D))
        
        self.res_block_3 = torch.nn.Sequential(
            ME.MinkowskiBatchNorm(128),
            MN.MinkowskiReLU(),
            ME.MinkowskiConvolution(
                in_channels=128,
                out_channels=128,
                kernel_size=3,
                stride=1,
                dimension=D),
            ME.MinkowskiBatchNorm(128),
            MN.MinkowskiReLU(),
            ME.MinkowskiConvolution(
                in_channels=128,
                out_channels=128,
                kernel_size=3,
                stride=1,
                dimension=D))
        
        self.conv_3 = torch.nn.Sequential(
            ME.MinkowskiConvolution(
                in_channels=128,
                out_channels=256,
                kernel_size=3,
                stride=2,
                dimension=D))
        

        # downsample

        self.upsample_3 = torch.nn.Sequential(
            ME.MinkowskiConvolutionTranspose(
                in_channels=256,
                out_channels=128,
                kernel_size=3,
                stride=2,
                dimension=D))
        
        self.conv_n_2 = torch.nn.Sequential(
            ME.MinkowskiConvolution(
                in_channels=256,
                out_channels=128,
                kernel_size=3,
                stride=1,
                dimension=D),
            ME.MinkowskiBatchNorm(128),
        )
        
        self.res_block_4 = torch.nn.Sequential(
            ME.MinkowskiBatchNorm(128),
            MN.MinkowskiReLU(),
            ME.MinkowskiConvolution(
                in_channels=128,
                out_channels=128,
                kernel_size=3,
                stride=1,
                dimension=D),
            ME.MinkowskiBatchNorm(128),
            MN.MinkowskiReLU(),
            ME.MinkowskiConvolution(
                in_channels=128,
                out_channels=128,
                kernel_size=3,
                stride=1,
                dimension=D))
        
        #
        self.upsample_2 = torch.nn.Sequential(
            ME.MinkowskiConvolutionTranspose(
                in_channels=128,
                out_channels=64,
                kernel_size=3,
                stride=2,
                dimension=D))
        
        self.conv_n_3 = torch.nn.Sequential(
            ME.MinkowskiConvolution(
                in_channels=128,
                out_channels=64,
                kernel_size=3,
                stride=1,
                dimension=D),
            ME.MinkowskiBatchNorm(64),
        )
        
        self.res_block_5 = torch.nn.Sequential(
            ME.MinkowskiBatchNorm(64),
            MN.MinkowskiReLU(),
            ME.MinkowskiConvolution(
                in_channels=64,
                out_channels=64,
                kernel_size=3,
                stride=1,
                dimension=D),
            ME.MinkowskiBatchNorm(64),
            MN.MinkowskiReLU(),
            ME.MinkowskiConvolution(
                in_channels=64,
                out_channels=64,
                kernel_size=3,
                stride=1,
                dimension=D))
        
        #
        self.upsample_1 = torch.nn.Sequential(
            ME.MinkowskiConvolutionTranspose(
                in_channels=64,
                out_channels=32,
                kernel_size=3,
                stride=2,
                dimension=D))
        
        self.conv_n_4 = torch.nn.Sequential(
            ME.MinkowskiConvolution(
                in_channels=64,
                out_channels=32,
                kernel_size=3,
                stride=1,
                dimension=D),
            ME.MinkowskiBatchNorm(32),
        )
        
        self.res_block_6 = torch.nn.Sequential(
            ME.MinkowskiBatchNorm(32),
            MN.MinkowskiReLU(),
            ME.MinkowskiConvolution(
                in_channels=32,
                out_channels=32,
                kernel_size=3,
                stride=1,
                dimension=D),
            ME.MinkowskiBatchNorm(32),
            MN.MinkowskiReLU(),
            ME.MinkowskiConvolution(
                in_channels=32,
                out_channels=32,
                kernel_size=3,
                stride=1,
                dimension=D))
        
        # final layer
        
        self.conv_4 = torch.nn.Sequential(
            ME.MinkowskiConvolution(
                in_channels=32,
                out_channels=out_nchannel,
                kernel_size=1,
                stride=1,
                dimension=D))

        

    def forward(self, x):
        #print("x.size:", x.size())

        out = self.conv_n_1(x)
        out_1 = self.res_block_1(out)
        #print(out_1.size())

        out = self.conv_1(out_1)
        out_2 = self.res_block_2(out)
        #print(out_2.size())

        out = self.conv_2(out_2)
        out_3 = self.res_block_3(out)
        #print(out_3.size())

        out = self.conv_3(out_3)
        out = self.upsample_3(out)
        out = ME.cat(MF.relu(out_3), out)
        #print(out.size())

        out = self.conv_n_2(out)
        out = self.res_block_4(out)
        out = self.upsample_2(out)
        out = ME.cat(MF.relu(out_2), out)
        #print(out.size())

        out = self.conv_n_3(out)
        out = self.res_block_5(out)
        out = self.upsample_1(out)
        out = ME.cat(MF.relu(out_1), out)
        #print(out.size())

        out = self.conv_n_4(out)
        out = self.res_block_6(out)
        out = self.conv_4(out)
        #print(out.size())

        return out


In [ ]:

import gc

gc.collect()
from torch._C import device
import torch.utils.data as utils_data
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import jaccard_score

from torch.nn import functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model     = ResUNet(6, 13, D= 3).to(device)

model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/model10_segv3.pth'))

criterion = torch.nn.CrossEntropyLoss()


outputs_list=[]

color_labels = {0:   [0,255,0],
                1:   [0,0,255],
                2:   [0,255,255],
                3:   [255,255,0],
                4:   [255,0,255],
                5:   [100,100,255],
                6:   [200,200,100],
                7:   [170,120,200],
                8:   [255,0,0],
                9:   [200,100,100],
                10:  [10,200,100],
                11:  [200,200,200],
                12:  [50,50,50]}

# fout = open('/content/drive/MyDrive/CS674_Project/vis_objects/test_pred_quant2.obj', 'w')



test_loss = 0

with torch.no_grad():
  js = 0

  for i, batch in enumerate(testloader):

    pos_batch, x_batch, y_batch = batch['pos'], batch['x'], batch['y']
    
    pos_batch = torch.cat((torch.ones(pos_batch.size()[0], 1)*i, pos_batch),1)
    minimum_pos_x = torch.min(pos_batch[:,1])
    maximum_pos_x = torch.max(pos_batch[:,1])
    minimum_pos_y = torch.min(pos_batch[:,2])
    maximum_pos_y = torch.max(pos_batch[:,2])
    minimum_pos_z = torch.min(pos_batch[:,3])
    maximum_pos_z = torch.max(pos_batch[:,3])
    #print("batch: ",i)
    #print("poslimites x,y,z : ",":x:",minimum_pos_x.item(),':',maximum_pos_x.item(),":y:",minimum_pos_y.item(),':',maximum_pos_y.item(),":z:",minimum_pos_z.item(),':',maximum_pos_z.item())
    
    coords, feats, labels = ME.utils.sparse_quantize(pos_batch.cpu(), x_batch.cpu(), y_batch.cpu(), quantization_size=0.1, device='cuda')
    input = ME.SparseTensor(feats, coords, device=device)
  
    outputs = model(input)

    outputs = outputs.F
    loss = criterion(outputs.to(device), labels.long().to(device))
    test_loss +=  loss.item()

    indices = torch.argmax(F.softmax(outputs,dim=1), dim=1).cpu()
    lbl = labels.cpu()
    js += jaccard_score(lbl, indices, labels=np.arange(13), average='weighted', zero_division=0.0)
    outputs_list.append(indices)

    #print(indices.size()[0])
    pos = np.array(coords.cpu())
    y = indices
    pos1 = coords
    minimum_pos_x = torch.min(pos1[:,1])
    maximum_pos_x = torch.max(pos1[:,1])
    minimum_pos_y = torch.min(pos1[:,2])
    maximum_pos_y = torch.max(pos1[:,2])
    minimum_pos_z = torch.min(pos1[:,3])
    maximum_pos_z = torch.max(pos1[:,3])
    #print("coords limits x,y,z : ",":x:",minimum_pos_x.item(),':',maximum_pos_x.item(),":y:",minimum_pos_y.item(),':',maximum_pos_y.item(),":z:",minimum_pos_z.item(),':',maximum_pos_z.item())
    
    # for j in range(indices.size()[0]):
  
    #     color = color_labels[y[j].item()]
    #     fout.write('v %f %f %f %d %d %d\n' % (pos[j][1] * 28.473, pos[j][2] * 21.591, pos[j][3] * 4.593, color[0], color[1], color[2]))



print('Testing loss:', test_loss/(i+1), '| mIoU:', js/(i+1))

# fout.close()


Testing loss: 1.1927417806253913 | mIoU: 0.5465487053332491
